In [1]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM
import numpy as np

# --- 1. Get Data ---
today = date.today()
d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=5000)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('AAPL',
                  start=start_date,
                  end=end_date,
                  progress=False)

data["Date"] = data.index






/tmp/ipython-input-3373462618.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('AAPL',


In [3]:
# --- 4. Prepare Data for LSTM ---
x = data[["Open", "High", "Low", "Volume"]]
y = data["Close"]
x = x.to_numpy()
y = y.to_numpy()
y = y.reshape(-1, 1)

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

print(f"\nxtrain shape: {xtrain.shape}")
print(f"xtest shape: {xtest.shape}")
print(f"ytrain shape: {ytrain.shape}")
print(f"ytest shape: {ytest.shape}")


# --- 5. Build LSTM Model ---
# Note: LSTM input_shape is (timesteps, features).
# Your data shape is (samples, features).
# We need to reshape xtrain and xtest to be 3D: (samples, timesteps, features)
# For this model, we'll assume 1 timestep.
xtrain = np.reshape(xtrain, (xtrain.shape[0], 1, xtrain.shape[1]))
xtest = np.reshape(xtest, (xtest.shape[0], 1, xtest.shape[1]))

print(f"Reshaped xtrain shape: {xtrain.shape}")
print(f"Reshaped xtest shape: {xtest.shape}")


model = Sequential()
# Input shape is (timesteps, features) -> (1, 4)
model.add(LSTM(128, return_sequences=True, input_shape=(xtrain.shape[1], xtrain.shape[2])))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()


xtrain shape: (2754, 4)
xtest shape: (689, 4)
ytrain shape: (2754, 1)
ytest shape: (689, 1)
Reshaped xtrain shape: (2754, 1, 4)
Reshaped xtest shape: (689, 1, 4)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1, 128)         │        68,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,625 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 119,155 (465.45 KB)

 Trainable params: 119,155 (465.45 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# --- 6. Compile and Train Model ---
model.compile(optimizer='adam', loss='mean_squared_error')
print("\n--- Starting Model Training ---")
model.fit(xtrain, ytrain, batch_size=1, epochs=30)
print("--- Model Training Complete ---")


# --- 7. Test the Model ---
# --- FIX IS HERE ---
# Corrected the comment to match the features: Open, High, Low, Volume
# features = [Open, High, Low, Volume]
features = np.array([[177.089996, 180.419998, 177.070007, 74919600]])

# We must reshape the test features just like we reshaped the training data
features = np.reshape(features, (features.shape[0], 1, features.shape[1]))

print("\n--- Predicting ---")
prediction = model.predict(features)
print(f"Predicted Close Price: {prediction[0][0]}")


--- Starting Model Training ---
Epoch 1/30
2754/2754 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - loss: 6365.0674
Epoch 2/30
2754/2754 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - loss: 5299.3252
Epoch 3/30
2754/2754 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - loss: 5301.3750
Epoch 4/30
2754/2754 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 5482.8281
Epoch 5/30
2754/2754 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - loss: 5509.2847
Epoch 6/30
2754/2754 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 5324.1270
Epoch 7/30
2754/2754 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 5351.0347
Epoch 8/30
2754/2754 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - loss: 5119.0649
Epoch 9/30
2754/2754 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 5394.1602
Epoch 10/30
2754/2754 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 5500.5850
Epoch 11/30
2754/2754 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 5264.5767
Epoch 12/30
2754/2754 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 5377.8735
Epoch 13/30
2754/2754 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 5289.7930
Epoch 14/30
27